In [1]:
FOLDER_ROOT = 'F:\Github'

In [61]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree

import sys
sys.path.append(FOLDER_ROOT)

from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score, f1_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer

from joblib import Parallel, delayed

import numpy as np

import pandas as pd

from os import cpu_count

In [9]:
df_stream = pd.read_csv('https://drive.google.com/u/0/uc?id=1-t29PMXa1zq0r5yxf5yV7ctdBdxduD_b&export=download',
                        engine='c', low_memory=True, memory_map=True)

X = df_stream[df_stream.columns[:-1]]
y = df_stream['target']

In [11]:
print('List columns:')
print(list(df_stream.columns))
print('Labels:')
print(set(y))

List columns:
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', 'target']
Labels:
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [17]:
X_train = X[:1000].values
y_train = y[:1000].values
X_test = X[1000:2000].values
y_test = y[1000:2000].values

y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

clf_rf = RandomForestClassifier(n_estimators=100,
                                criterion='entropy',
                                max_features='auto',
                                bootstrap=True,
                                oob_score=True,
                                random_state=42,
                                warm_start=True,
                                n_jobs=-1
)
clf_rf = clf_rf.fit(X_train, y_train)
print("\nRandom Forest (with 100 trees) trained.")
print('Score within of training data = ', clf_rf.score(X_train, y_train))


Random Forest (with 100 trees) trained.
Score within of training data =  1.0


In [18]:
dt0 = clf_rf.estimators_[0]

y_pred = dt0.predict(X_test)

In [64]:
y_proba = dt0.predict_proba(X_test)
print('Shape yproba: ', y_proba.shape)

Shape yproba:  (1000, 10)


In [21]:
print('Test: ', list(y_test[:10]))
print('Pred: ', list(y_pred[:10]))

Test:  [9.0, 1.0, 7.0, 8.0, 6.0, 2.0, 1.0, 7.0, 7.0, 3.0]
Pred:  [7.0, 7.0, 4.0, 9.0, 6.0, 2.0, 1.0, 3.0, 1.0, 4.0]


In [30]:
plabels = set(y)
plabels = (list(plabels),)
print(plabels)
plabels = np.array(plabels)

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],)


In [63]:
lb = LabelBinarizer()
lb.fit(y)
print('LB Classes: ', lb.classes_)

yt = lb.transform(y_test)
yp = lb.transform(y_pred)

print('Shape yt: ', yt.shape)
print('Shape yp: ', yp.shape)

LB Classes:  [0 1 2 3 4 5 6 7 8 9]
Shape yt:  (1000, 10)
Shape yp:  (1000, 10)


In [93]:
lb.transform([9])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [65]:
accuracy_score(y_test, y_pred)

0.317

In [66]:
accuracy_score(yt, yp)

0.317

In [67]:
cohen_kappa_score(y_test, y_pred)

0.24081924828878642

In [72]:
#cohen_kappa_score(yt, yp, labels=y_test, weights='linear') # não rola

In [74]:
f1_score(y_test, y_pred, average='weighted')

0.3187039675293989

In [77]:
f1_score(yt, yp, average='weighted')

0.3187039675293989

In [79]:
balanced_accuracy_score(y_test, y_pred)

0.31536282460964365

In [81]:
#balanced_accuracy_score(yt, yp, adjusted=True) # não rola

In [83]:
precision_score(y_test, y_pred, average='weighted')

0.3225460379751473

In [85]:
precision_score(yt, yp, average='weighted')

0.3225460379751473

In [87]:
recall_score(y_test, y_pred, average='weighted')

0.317

In [88]:
recall_score(yt, yp, average='weighted')

0.317

In [91]:
#roc_auc_score(y_test, y_pred, average='weighted', multi_class='ovr') # não funciona

In [90]:
roc_auc_score(yt, yp, average='weighted', multi_class='ovr')

0.6205451567943195

In [92]:
roc_auc_score(yt, yp, average='weighted', multi_class='ovo')

0.6205451567943195